In [35]:
# imports

import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests
import gradio as gr # oh yeah!


In [36]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('ENDPOINT')
version = os.getenv('VERSION')
deployment = os.getenv('DEPLOYMENT_4_1nano')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# elif not api_key.startswith("sk-proj-"):
#     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

# Check the endpoint

if  not endpoint:
    print("No endpoint  was found")
else:
    print("endpoint found and looks good so far!")


API key found and looks good so far!
endpoint found and looks good so far!


In [37]:
client = AzureOpenAI(
    azure_endpoint=endpoint, 
    api_key=api_key,
    api_version=version
)

Read the file

In [ ]:
import re

file_name = "relationship2.txt"

with open(file_name, "r", encoding="utf-8") as f:
    full_text = f.read()

In [ ]:
def generic_response(user_prompt, system_prompt, model=deployment):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
    )
    return response.choices[0].message.content.strip()

First thing is to transalte it to english and add before each part the week and day according to what's it says in the format:
Week X Day Y

In [ ]:
def translate_week_episodes(text):
    system_prompt = "You are a helpful assistant that gets text in Hebrew " \
    "and translates it to English and can do requested changes to the text."

    user_prompt ="Translate the following text to English and according to the text, add the line Week X Day Y "
    user_prompt += "\n\n" + text

    # Call the Azure OpenAI API
    response = generic_response(user_prompt, system_prompt)
    return response

getting full text with separation by pattern - divide it to different days

In [ ]:
def divide_episodes(text): 
    # this is a regex pattern to split the text into parts
    pattern = r'(Week\s+\d+\s+Day\s+\d+)'
    parts = re.split(pattern, text)

    days = []
    for i in range(1, len(parts), 2):
        title = parts[i].strip()
        content = parts[i + 1].strip()
        days.append((title, f"{title}\n{content}"))
    return days


Each day will get a title and a summarization

In [13]:
system_prompt_ = "You are a helpful assistant. \
    For each Week and Day, summarize the key information briefly."

def user_prompt_for_title(day_text, z):
    user_prompt = f"Please summarize the following content into a single line \
    to be used as a podcast episode title, \
        in the format: Week X: Day Y – NAME (Z). \
            X is the Week number, Y is the day number \
            and Z is the value {z}. \
            NAME is the episode title you create.\n"
    user_prompt += day_text
    return user_prompt

def user_prompt_for_summary(day_text):
    user_prompt = f"Please summarize the following content into a single line \
    to be used as a podcast episode summary."
    user_prompt += day_text
    return user_prompt

In [14]:
results = []
podcast_episode_num = 8
# Loop through the days and summarize each one
for title, day_text in days:
    print(f"📤 Processing {title}...")
    
    try:
        day_title = generic_response(
            user_prompt_for_title(day_text, podcast_episode_num),
            system_prompt_
        )
        summary = generic_response(
            user_prompt_for_summary(day_text),
            system_prompt_
        )
    except Exception as e:
        print(f"⚠️ Error with {title}: {e}")
        day_title = None
        summary = None

    results.append({
        "day": day_title,
        "response": summary
    })
    podcast_episode_num += 1
    print(f"📥 Received: {day_title}:\n{summary}")

📤 Processing Week 2 Day 1...
📥 Received: Week 2: Day 1 – Building Your Partner's "Love Map" (8):
Week 2 Day 1: Learn how to build a deeper "Love Map" by understanding your partner’s inner world, feelings, dreams, and influences through self-reflection and conversation.
📤 Processing Week 2 Day 2...
📥 Received: Week 2: Day 2 – Exploring Shared Visions of the Future (9):
Explore the importance of discussing and sharing each partner’s vision for the future to strengthen connection and understanding over time.
📤 Processing Week 2 Day 3...
📥 Received: Week 2: Day 3 – Building Deeper Connection Through Small Acts:
Discover how focusing on small, meaningful gestures and understanding your partner's positive triggers can strengthen your relationship.
📤 Processing Week 2 Day 4...
📥 Received: Week 2: Day 4 – Understanding and Supporting Your Girlfriend’s Stress (11):
Week 2 Day 4: Learn to empathetically understand and support your girlfriend’s daily stresses to strengthen your bond.
📤 Processing

Text to Speech

In [21]:
# Your Azure info
region = os.getenv('AZURE_TTS_REGION')
key = os.getenv('AZURE_TTS_API_KEY')

# Set the endpoint and headers

endpoint = f"https://{region}.tts.speech.microsoft.com/cognitiveservices/v1"
headers = {
    "Ocp-Apim-Subscription-Key": key,
    "Content-Type": "application/ssml+xml",
    "X-Microsoft-OutputFormat": "audio-16khz-32kbitrate-mono-mp3",
    "User-Agent": "myTTSApp"
}

In [33]:
def create_ssml(text, voice="en-US-JennyNeural"):
    ssml = f"""
    <speak version='1.0' xml:lang='en-US'>
    <voice name='{voice}'>
        {text}
    </voice>
    </speak>
    """
    return ssml

def create_audio_file(ssml, filename):
    # Make the POST request
    response = requests.post(endpoint, headers=headers, data=ssml.encode('utf-8'))

    # Save to file
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Audio saved as {filename}")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)


In [30]:
text = "week 1 day 7,  creating shared meaning"
ssml = create_ssml(text,"en-US-AndrewNeural")
create_audio_file(ssml, "output.mp3")


Audio saved as output.mp3


In [34]:
# Loop through the results and create audio files 
for title, day_text in days:
    print(f"📤 Processing {title}...")
    
    try:
        ssml = create_ssml(day_text, "en-US-AndrewNeural")
        filename = f"{title}.mp3"
        create_audio_file(ssml, filename)
    except Exception as e:
        print(f"⚠️ Error with {title}: {e}")  

📤 Processing Week 2 Day 1...
Audio saved as Week 2 Day 1.mp3
📤 Processing Week 2 Day 2...
Audio saved as Week 2 Day 2.mp3
📤 Processing Week 2 Day 3...
Audio saved as Week 2 Day 3.mp3
📤 Processing Week 2 Day 4...
Audio saved as Week 2 Day 4.mp3
📤 Processing Week 2 Day 5...
Audio saved as Week 2 Day 5.mp3
📤 Processing Week 2 Day 6...
Audio saved as Week 2 Day 6.mp3
📤 Processing Week 2 Day 7...
Audio saved as Week 2 Day 7.mp3


In [46]:

view = gr.Interface(
    fn=translate_episode,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
